In [ ]:
# notebooks/HATR_Recognition.ipynb

# Importer les bibliothèques nécessaires
import numpy as np
import tensorflow as tf
from transformers import ViTForImageClassification, ViTFeatureExtractor
from sklearn.model_selection import train_test_split

In [ ]:
# Charger les données prétraitées
images = np.load('/content/data/images.npy')
labels = np.load('/content/data/labels.npy')

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Préparer les données pour le modèle ViT
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
X_train = feature_extractor(images=X_train, return_tensors='tf')['pixel_values']
X_test = feature_extractor(images=X_test, return_tensors='tf')['pixel_values']

# Convertir les étiquettes en format de tenseurs
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [ ]:
# Charger le modèle ViT pré-entraîné
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=NUM_CLASSES)

# Compiler le modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Sauvegarder le modèle
model.save_pretrained('/content/models/hatr_model')